# Introduction to Synthetic Data Generation with Llama 3.1 405B for Fine Tuning Smaller Models

In this notebook, we will walk you through the process of utilizing a larger language model (LLM) like Llama 3.1 405B, Meta AI's latest and most advanced model, to create  a dataset for instruction fine-tuning. This dataset will be used to fine-tune a smaller model, such as Llama 3 8B.

By leveraging the capabilities of Llama 3.1 405B, we can generate high-quality, concise training data that enhances the performance of smaller models. This approach is particularly useful for tasks that require detailed and specific instructions.

Before we begin, ensure that you have access to Llama 3.1 405B, which is now available on Amazon SageMaker Jumpstart. You can find the dataset we will be using [here](https://huggingface.co/datasets/deepmind/aqua_rat).

You can run the notebook on an Amazon SageMaker Studio notebook, or a SageMaker notebook instance without manually setting your aws credentials.

Let's get started!

### Amazon SageMaker JumpStart

![Alt text](imgs/jumpstart-overview-img1.png "SageMaker JumpStart Overview")

**Amazon SageMaker JumpStart** is a powerful feature within Amazon SageMaker designed to help you quickly get started with LLMs by providing access to a wide range of pre-trained foundation models (FM). We'll be using this for deploying and fine tuning our models.

Key Features
- **Pre-trained Models**: SageMaker JumpStart provides a variety of pre-trained models from different model providers (Llama, Mistral, Cohere, Stablity) for different problem types, enabling you to start your machine learning projects without the need to build models from scratch.

- **Training and Tuning**: With a few clicks, you can train and fine-tune these models to better fit your specific data and use case before deploying them.

- **Solution Templates**: JumpStart offers solution templates that automatically set up the necessary infrastructure for common use cases, streamlining the deployment process.

### Llama 3.1 405B Model

Llama 3.1 405B is the largest model in the family of Llama 3.1 models. Llama 3.1 model family is a collection of pre-trained and instruction-tuned LLMs which already includes 8B and 70B parameter sizes. Llama 3.1 405B comes with new capabilities including multi-language support and a 128k context window. These models are stronger overall capabilities and are ideal for content creation, conversational AI, language understanding, research and development (R&D), and enterprise applications.


### Llama 3 8B Model

LLama 3 8B is an LLM with 8 billion parameters designed to deliver high performance across a variety of tasks while maintaining cost efficiency. This model is particularly advantageous for developers and organizations looking to implement advanced AI capabilities without the need for extensive computational resources. LLaMA 3 8B is optimized for dialogue and other interactive applications, demonstrating strong performance in benchmarks such as MMLU, AGIEval, and CommonSenseQA, where it outperforms many open-source models of similar size. Its ability to run on more affordable hardware highlights its potential for cost-effective deployment in real-time applications like chatbots and customer support systems.



### Prequisites
 In order to follow along in this notebook, you'll need access to the following:

 - An AWS account with SageMaker endpoint capacity for an ml.p4de instance type. You can find more information about how to request a service limit increase [here](https://docs.aws.amazon.com/servicequotas/latest/userguide/request-quota-increase.html).

 - An [AWS Identity and Access Management (IAM)](https://aws.amazon.com/iam/) role to access SageMaker. To learn more about how IAM works with SageMaker, refer to [Identity and Access Management for Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/security-iam.html).
 
 - Access to SageMaker Studio or a SageMaker notebook instance or an interactive development environment (IDE) such as PyCharm or Visual Studio Code. We recommend using SageMaker Studio for straightforward deployment and inference.


### In this notebook, we perform the following high level steps: 

1. Demonstrate the ability to use a large model, in this case, `Llama 3.1 405B Model` to generate labels that can be used to fine tune a smaller model to generate more accurate responses, saving on inference cost and latency.

1. We deploy a `Llama3-8b instruct` model and generate inferences on a `deepmind/aqua_rat` dataset.

1. Deploy and leverage the capabilities of the new `Llama 3.1 405B Model` to generate labels and corresponding data to be used to fine tune `Llama3-8b instruct`

1. Test the fine tuned `Llama3-8b instruct` model and rung the model against the same questions to showcasw the increase in response quality.

In [2]:
# Import necessary libraries
import logging
import sagemaker
from sagemaker import get_execution_role
from sagemaker.jumpstart.model import JumpStartModel
import json
from IPython.core.display import display, HTML
import boto3
from botocore.exceptions import ClientError
import os


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Preferences/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/bustils/Library/Preferences/sagemaker/config.yaml


/var/folders/n6/y6pw1tpj3kqg9jj7ly3w_n4m0000gr/T/ipykernel_25621/1469671555.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# We'll use this function to call inference on our deployed models
def run_inference(predictor, example_payloads):
    for payload in example_payloads:
        response = predictor.predict(payload)
        response = response[0] if isinstance(response, list) else response
        # print("Input:\n", payload.body, end="\n\n")
        print("Input:\n", payload["inputs"], end="\n\n")
        print("Output:\n", response["generated_text"].strip(), end="\n\n\n")


## Dataset Exploration and Preparation

In this section, we will explore a dataset from the Hugging Face Hub using the HF Datasets library. Hugging Face provides a vast collection of datasets for various tasks in natural language processing (NLP), computer vision, and audio processing. This exploration will help us understand the structure, features, and contents of the dataset, enabling us to prepare it for training and evaluation in our machine learning models. The [deepmind/aqua_rat](https://huggingface.co/datasets/deepmind/aqua_rat) dataset is a large-scale collection of approximately 100,000 algebraic word problems, each accompanied by a detailed natural language rationale explaining the solution process. This dataset is designed to train and evaluate models that not only generate the correct answer but also provide a step-by-step explanation, making it ideal for tasks requiring mathematical reasoning and natural language understanding.

In [5]:

# Import the necessary functions from the datasets library
from datasets import load_dataset, DatasetDict

# Load the AQUA-RAT dataset from the Hugging Face Hub
dataset_name = "deepmind/aqua_rat"
dataset = load_dataset(dataset_name)

# Display basic information about the dataset
print(f"Dataset: {dataset_name}")
print(dataset)

# Display the dataset's features
print("\nDataset Features:")
print(dataset['train'].features)

# Display a few examples from the dataset
print("\nSample Examples:")
for i in range(3):
    print(dataset['train'][i])

# Display the number of examples in each split
print("\nNumber of Examples in Each Split:")
for split in dataset.keys():
    print(f"{split}: {len(dataset[split])} examples")

# Extract 20 questions from the dataset
questions = dataset['train'].select(range(20))['question']

# Display the first 20 questions
print("\nFirst 20 Questions:")
for i, question in enumerate(questions):
    print(f"{i+1}: {question}")

[2024-07-23 16:18:29,994] p25621 {config.py:58} INFO - PyTorch version 1.13.0 available.


Dataset: deepmind/aqua_rat
DatasetDict({
    train: Dataset({
        features: ['question', 'options', 'rationale', 'correct'],
        num_rows: 97467
    })
    test: Dataset({
        features: ['question', 'options', 'rationale', 'correct'],
        num_rows: 254
    })
    validation: Dataset({
        features: ['question', 'options', 'rationale', 'correct'],
        num_rows: 254
    })
})

Dataset Features:
{'question': Value(dtype='string', id=None), 'options': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'rationale': Value(dtype='string', id=None), 'correct': Value(dtype='string', id=None)}

Sample Examples:
{'question': "Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?", 'options': ['A)21', 'B)21.5', 'C)22', 'D)22.5', 'E)23'], 'rationale': 'If Q complete x kilometers, then P 

### Deploying Llama 3 8B Instruct

In this section, we will deploy the base, pre-trained LLama 3 8B model and test it against a subset of our dataset to evaluate its responses compared to the larger LLama 3 70B model. Initially, we expect the smaller model to produce lower-quality responses. By identifying these deficiencies, we can generate high-quality synthetic data using the 70B model and subsequently fine-tune the 8B model. This process aims to demonstrate the improvement in response quality after fine-tuning the 8B model with the enhanced dataset.

> You'll need a `g5.12xlarge` instance for endpoint usage to deploy this model.

In [9]:
# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Specify the role ARN directly
role = get_execution_role(sagemaker_session=sagemaker_session)

# Select a model ID and version
llama_3_8b_model_id = "meta-textgeneration-llama-3-8b-instruct" # Replace with your chosen model ID

# If your selected model is gated, you will need to set accept_eula to True to accept the model end-user license agreement (EULA).
accept_eula = False

# Deploy the model to a SageMaker endpoint
llama_3_8b_model = JumpStartModel(model_id=llama_3_8b_model_id,role=role)
llama_3_8b_predictor = llama_3_8b_model.deploy(accept_eula=accept_eula)

# example_payloads = llama_3_8b_model.retrieve_all_examples() # uncomment if you want to preloaded examples instead

question = questions[0]

example_payloads = [
    {
        "inputs": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "parameters": {
            "max_new_tokens": 256,
            "top_p": 0.9,
            "temperature": 0.6,
            "details": True,
            "stop": "<|eot_id|>"
        }
    }
]

print("Running inference with LLama 3 8B model:\n")
run_inference(llama_3_8b_predictor, example_payloads)


[2024-07-23 16:23:33,426] p25621 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


Session(region_name='us-west-2')


[2024-07-23 16:23:34,531] p25621 {session.py:5807} WARNING - Couldn't call 'get_role' to get Role ARN from role name mac-user-cli to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::762797705265:user/mac-user-cli, therefore it cannot be used as a SageMaker execution role

## Deploying LLama 3.1 405B Instruct

In this section, we will deploy the LLama 3.1 405B model to compare its responses with those of the smaller LLama 3 8B model. This deployment will allow us to evaluate the performance differences and identify areas where the 8B model's responses can be improved. By analyzing the responses from the 405B model, we can generate high-quality data to fine-tune the 8B model, enhancing its accuracy and effectiveness for domain-specific tasks.

> You'll need a 'p5.48xlarge' instance for endpoint usage to deploy this model.

In [83]:
# Select a model ID and version
llama_3_1_405b_model_id = "meta-textgeneration-llama-3-1-405b-instruct" # Replace with your chosen model ID

# If your selected model is gated, you will need to set accept_eula to True to accept the model end-user license agreement (EULA).
accept_eula = False

# Deploy the model to a SageMaker endpoint
llama_3_1_405b_model = JumpStartModel(model_id=llama_3_1_405b_model_id,role=role)
llama_3_1_405b_predictor = llama_3_1_405b_model.deploy(accept_eula=accept_eula)

# example_payloads = model.retrieve_all_examples()

question = questions[1]

example_payloads = [
    {
        "inputs": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "parameters": {
            "max_new_tokens": 256,
            "top_p": 0.9,
            "temperature": 0.6,
            "details": True
        }
    }
]


# Test the deployed endpoint
print("Running inference with LLama 3 405B model:\n")
response = llama_3_1_405b_predictor.predict(example_payloads[0])

No instance type selected for inference hosting endpoint. Defaulting to ml.p4d.24xlarge.
[2024-07-17 22:56:03,353] p54 {model.py:201} INFO - No instance type selected for inference hosting endpoint. Defaulting to ml.p4d.24xlarge.
[2024-07-17 22:56:03,370] p54 {session.py:3872} INFO - Creating model with name: meta-textgeneration-llama-3-70b-instruc-2024-07-17-22-56-03-353
[2024-07-17 22:56:04,235] p54 {session.py:5632} INFO - Creating endpoint-config with name meta-textgeneration-llama-3-70b-instruc-2024-07-17-22-56-03-357
[2024-07-17 22:56:04,590] p54 {session.py:4478} INFO - Creating endpoint with name meta-textgeneration-llama-3-70b-instruc-2024-07-17-22-56-03-357


------------!Running inference with LLaMA 3 70B model:



["A nice problem!\n\nSince line k passes through the origin, its equation is of the form y = mx, where m is the slope. We are given that the slope is 1/5, so the equation of the line is:\n\ny = (1/5)x\n\nWe are also given two points on the line: (x, 1) and (5, y). We can use these points to find the values of x and y.\n\nFirst, let's use the point (x, 1). Substitute x into the equation of the line:\n\n1 = (1/5)x\n\nMultiply both sides by 5 to get:\n\n5 = x\n\nSo, x = 5.\n\nNow, let's use the point (5, y). Substitute 5 into the equation of the line:\n\ny = (1/5)(5)\n\ny = 1\n\nSo, y = 1.\n\nTherefore, the values of x and y are 5 and 1, respectively."]

## Using Llama 3.1 405B for Data Labeling/Generation

In this section, we will leverage the LLama 3.1 405B model to generate high-quality synthetic data for fine-tuning the LLama 3 8B model. By using the 405B model to generate responses to domain-specific prompts, we can create a labeled dataset that will be used to fine-tune the 8B model, improving its accuracy and effectiveness in specific tasks.

In [ ]:
# Function to run inference and generate synthetic data
def generate_synthetic_data(predictor, example_payloads):
    synthetic_data = []
    for payload in example_payloads:
        response = predictor.predict(payload)
        response = response[0] if isinstance(response, list) else response
        synthetic_data.append({
            "instruction": payload["inputs"],
            "response": response["generated_text"].strip()
        })
    return synthetic_data



# Generate synthetic data using LLama 3.1 405B
synthetic_data = generate_synthetic_data(llama_3_1_405b_predictor, example_payloads)

# Save the synthetic data to a JSONL file
with open('synthetic_data.jsonl', 'w') as f:
    for entry in synthetic_data:
        f.write(json.dumps(entry) + '\n')


# Initialize the S3 client
s3 = boto3.client('s3')
bucket_name = '<<INSERT_BUCKET_NAME>>'  # Create a new bucket or use an existing one
subdirectory = 'llama-405b-synthetic-training-data'
train_data_location = f"s3://{bucket_name}/{subdirectory}"

files_to_upload = ['template.json','synthetic_data.jsonl']

# Upload the files to the specified subdirectory
for file_name in files_to_upload:
    file_path = file_name  # File is in the same directory as the notebook
    key_path = f"{subdirectory}/{file_name}"
    
    # Check if the file exists
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"No such file or directory: '{file_path}'")
    
    # Upload the file
    try:
        s3.upload_file(file_path, bucket_name, key_path)
        print(f"File {file_name} uploaded successfully to {key_path}.")
    except ClientError as e:
        print(f"Error uploading file {file_name}: {e}")

## (Optional) Bedrock Example

In [ ]:
import boto3
from botocore.exceptions import ClientError

# Initialize the Bedrock client
client = boto3.client("bedrock-runtime", region_name="us-west-2")

# Set the model ID, e.g., Llama 3 8b Instruct.
model_id = "meta.llama3-1-8b-instruct-v1:0"

# Function to run inference and generate synthetic data using Bedrock
def generate_synthetic_data(client, model_id, example_payloads):
    synthetic_data = []
    for payload in example_payloads:
        user_message = payload["inputs"]
        conversation = [
            {
                "role": "user",
                "content": [{"text": user_message}],
            }
        ]
        try:
            # Send the message to the model, using a basic inference configuration.
            response = client.converse(
                modelId=model_id,
                messages=conversation,
                inferenceConfig={
                    "maxTokens": payload["parameters"]["max_new_tokens"],
                    "temperature": payload["parameters"]["temperature"],
                    "topP": payload["parameters"]["top_p"]
                },
            )

            # Extract the response text
            response_text = response["output"]["message"]["content"][0]["text"].strip()
            synthetic_data.append({
                "instruction": user_message,
                "response": response_text
            })
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
            continue  # Skip the current payload and continue with the next one

    return synthetic_data

# Example payloads
example_payloads = [
    {
        "inputs": "Describe the purpose of a 'hello world' program in one line.",
        "parameters": {
            "max_new_tokens": 256,
            "top_p": 0.9,
            "temperature": 0.6,
            "details": True,
            "stop": "<|eot_id|>"
        }
    }
]

# Generate synthetic data using Bedrock
synthetic_data = generate_synthetic_data(client, model_id, example_payloads)

# Print the synthetic data
for data in synthetic_data:
    print(f"Instruction: {data['instruction']}")
    print(f"Response: {data['response']}")

## Fine Tuning Llama 3 8B

In this section, we will dive deep into the process of fine-tuning the LLaMA 3 8B model to enhance its performance for specific tasks. Fine-tuning involves training the pre-trained model on custom datasets to adapt it to particular domains or applications. This process can be resource-intensive, but using techniques such as LoRA (Low Rank Adaptation) and QLoRA (Quantized LoRA) can significantly reduce the required computational resources and costs. We will explore how to set up and execute a fine-tuning job using SageMaker.

> You'll need a `g5.12xlarge` instance for endpoint usage to deploy this model.

In [136]:
from sagemaker.jumpstart.estimator import JumpStartEstimator

model_id, model_version = "meta-textgeneration-llama-3-8b-instruct", "*"


estimator = JumpStartEstimator(
    model_id=model_id,
    model_version=model_version,
    environment={"accept_eula": "true"},  # Please change {"accept_eula": "true"}
    disable_output_compression=True,
    instance_type="ml.g5.12xlarge",  # For Llama-3-70b, add instance_type = "ml.g5.48xlarge"
)
# By default, instruction tuning is set to false. Thus, to use instruction tuning dataset you use
estimator.set_hyperparameters(
    instruction_tuned="True", epoch="1", max_input_length="1024", chat_dataset="False"
)
estimator.fit({"training": train_data_location})

[2024-07-19 13:54:25,263] p54 {session.py:978} INFO - Creating training-job with name: meta-textgeneration-llama-3-8b-instruct-2024-07-19-13-54-25-260


2024-07-19 13:54:25 Starting - Starting the training job...
2024-07-19 13:54:25 Pending - Training job waiting for capacity...........................
2024-07-19 13:59:21 Pending - Preparing the instances for training...
2024-07-19 13:59:52 Downloading - Downloading input data...........................
2024-07-19 14:04:18 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-07-19 14:04:20,635 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-07-19 14:04:20,671 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-07-19 14:04:20,680 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-07-19 14:04:20,682 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-07-19 14:04:29,784 sagemaker-training-toolkit INF

## Testing the LLama 3 8B Fine Tuned Model 

In this section, we will evaluate the performance of the fine-tuned LLaMA 3 8B model to determine how well it has adapted to the specific tasks for which it was trained. Testing involves comparing the model's responses to a set of predefined questions or tasks against the baseline performance of the original, pre-trained model. This process helps us understand the improvements achieved through fine-tuning and identify any remaining areas for enhancement. By systematically examining the model's outputs, we can ensure that the fine-tuning process has effectively tailored the model to meet our specific requirements.

In [137]:
finetuned_predictor = estimator.deploy()

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.12xlarge.
[2024-07-19 14:34:50,076] p54 {model.py:201} INFO - No instance type selected for inference hosting endpoint. Defaulting to ml.g5.12xlarge.
[2024-07-19 14:34:50,165] p54 {session.py:3872} INFO - Creating model with name: meta-textgeneration-llama-3-8b-instruct-2024-07-19-14-34-50-079
[2024-07-19 14:34:50,838] p54 {session.py:5632} INFO - Creating endpoint-config with name meta-textgeneration-llama-3-8b-instruct-2024-07-19-14-34-50-076
[2024-07-19 14:34:51,179] p54 {session.py:4478} INFO - Creating endpoint with name meta-textgeneration-llama-3-8b-instruct-2024-07-19-14-34-50-076


--------------!

In [138]:
# Extract 4 questions, options, and their correct answers from the dataset
num_questions = 4
questions = dataset['train'].select(range(num_questions))['question']
options = dataset['train'].select(range(num_questions))['options']
correct_answers = dataset['train'].select(range(num_questions))['correct']

# Map the correct answer letter to the actual answer
def get_correct_answer(options, correct_letter):
    for option in options:
        if option.startswith(correct_letter):
            return option.split(')', 1)[1].strip()
    return None

actual_correct_answers = [get_correct_answer(opt, correct) for opt, correct in zip(options, correct_answers)]

# Define the inference parameters
params = {
    "max_new_tokens": 512,  # Increase this value to allow longer responses
    "top_p": 0.9,  # Adjust to introduce variability
    "temperature": 0.0,  # Adjust to introduce variability
    "details": True,
    "stop": "<|eot_id|>"
}

# Define the example payloads list
example_payloads = [
    {
        "inputs": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "parameters": params
    }
    for question in questions
]

# Function to run inference and collect results
def run_inference(predictor, example_payloads):
    results = []
    for payload in example_payloads:
        response = predictor.predict(payload)
        response = response[0] if isinstance(response, list) else response
        generated_text = response["generated_text"].strip()
        
        # Check if the response is truncated
        if generated_text.endswith("..."):
            generated_text += " [TRUNCATED]"
        
        results.append(generated_text)
    return results

# Run inference with both models
print("Running inference with fine-tuned LLama 3 8B model...\n")
results_fine_tuned_8b = run_inference(finetuned_predictor, example_payloads)

print("Running inference with LLama 3 8B model...\n")
results_8b = run_inference(llama_3_8b_predictor, example_payloads)


# Create a table of the outputs using HTML
table_html = """
<table>
    <tr>
        <th>Question</th>
        <th>Correct Answer</th>
        <th>Fine-tuned LLama 3 8B Output</th>
        <th>LLama 3 8B Output</th>
    </tr>
"""

for i in range(4):
    table_html += f"""
    <tr>
        <td>{questions[i]}</td>
        <td>{actual_correct_answers[i]}</td>
        <td>{results_fine_tuned_8b[i]}</td>
        <td>{results_8b[i]}</td>
    </tr>
    """

table_html += "</table>"

# Display the table using HTML
display(HTML(table_html))

Running inference with fine-tuned LLaMA 3 8B model...

Running inference with LLaMA 3 8B model...



Question,Correct Answer,Fine-tuned LLaMA 3 8B Output,LLaMA 3 8B Output
"Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?",23,"Let's say Friend Q's rate is x km/hr. Then Friend P's rate is x + 15% of x = 1.15x km/hr. Since they walk for the same amount of time, the distance they cover is the same. So, x * t = 1.15x * t, where t is the time they walk. So, x = 1.15x. So, x = 0. Therefore, x = 0. So, the distance covered by Friend P when they pass each other is 0. The answer is 0.",Let's say Friend Q's rate is x. Then Friend P's rate is 1.15x. The total distance is 43 km. So the time it takes for them to meet is 43 / (x + 1.15x) = 43 / 2.15x. The distance Friend P will have walked is 1.15x * (43 / 2.15x) = 20.5. The answer is 20.5.
"In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively?",5 and 1,"## Step 1: Since the line passes through the origin, we can write the equation of the line as y = (1/5)x. Step 2: We are given that the point (x, 1) is on the line, so we can substitute x for x and 1 for y in the equation of the line to get 1 = (1/5)x. Step 3: Solving for x, we get x = 5. Step 4: We are also given that the point (5, y) is on the line, so we can substitute 5 for x and y for y in the equation of the line to get y = (1/5)(5). Step 5: Solving for y, we get y = 1. Step 6: Therefore, the values of x and y are 5 and 1, respectively.","The slope of the line is 1/5, so the slope between the points (0,0) and (x,1) is 1/5. This means that $$\frac{1-0}{x-0}=\frac{1}{5}\Rightarrow x=5.$$Similarly, the slope between the points (5,y) and (0,0) is 1/5, so $$\frac{y-0}{5-0}=\frac{1}{5}\Rightarrow y=1.$$Therefore, the values of x and y are 5 and 1, respectively. Final Answer: The final answer is 5 and 1. I hope it is correct."
"For all numbers p and q, the operation @ is defined by p@q = p^2 - pq. If xy ≠ 0, then which of the following can be equal to zero? I. x@y II. (xy)@y III. x@(x + y)",I and II,"## Step 1: Define the operation @ for p and q as p@q = p^2 - pq.\n\n## Step 2: Given that xy ≠ 0, we need to find which of the options can be equal to zero.\n\n## Step 3: Option I: x@y = x^2 - xy = x(x - y). Since xy ≠ 0, x ≠ 0 or y ≠ 0. If x = 0, then x@y = 0. If y = 0, then x@y = x^2 ≠ 0. Therefore, x@y can be equal to zero.\n\n## Step 4: Option II: (xy)@y = (xy)^2 - xy(y) = x^2y^2 - xy^2 = x^2y(y - 1). Since xy ≠ 0, y ≠ 0 or y ≠ 1. If y = 1, then (xy)@y = x^2y - xy = x^2 - x = x(x - 1). If y ≠ 1, then (xy)@y = 0. Therefore, (xy)@y can be equal to zero.\n\n## Step 5: Option III: x@(x + y) = x^2 - x(x + y) = x^2 - x^2 - xy = -xy. Since xy ≠ 0, -xy ≠ 0. Therefore, x@(x + y) cannot be equal to zero.\n\nThe final answer is: I and II.","A nice algebraic puzzle! Let's analyze each option: I. x@y = x^2 - xy = x(x - y) = 0 This can be equal to zero if x = 0 or x = y. However, we are given that xy ≠ 0, so x cannot be zero. Therefore, x@y cannot be equal to zero. II. (xy)@y = (xy)^2 - (xy)y = x^2y^2 - xy^2 = x^2y(y - 1) = 0 This can be equal to zero if x = 0, y = 1, or y = 1. However, we are given that xy ≠ 0, so y cannot be 1. Therefore, (xy)@y cannot be equal to zero. III. x@(x + y) = x^2 - x(x + y) = x^2 - x^2 - xy = -xy This can be equal to zero if xy = 0, which is allowed since xy ≠ 0 is not a given condition. Therefore, the correct answer is III. x@(x + y)."
"Carl is facing very difficult financial times and can only pay the interest on a $10,000 loan he has taken. The bank charges him a quarterly compound rate of 4%. What is the approximate interest he pays annually?",$1600,"The interest rate is 4% per quarter. So the interest rate per year is 4 * 4 = 16%. 16% of 10,000 is 1600. The answer is 1600.","The interest rate is 4% per quarter. So the interest rate 

## Conclusion

In this notebook, we have successfully demonstrated the process of fine-tuning and evaluating the LLama 3 8B model using Amazon SageMaker JumpStart. By leveraging the advanced capabilities of the LLama 3.1 405B model, we generated high-quality synthetic data that served as a foundation for fine-tuning the smaller 8B model. This approach allowed us to enhance the performance of the LLaMA 3 8B model, tailoring it to specific domain tasks and improving its accuracy and effectiveness.

### Key Steps Accomplished:
1. **Dataset Exploration**: We explored a sample dataset to understand its structure and contents, preparing it for use in model training and evaluation.
2. **Data Generation with LLama 3.1 405B**: Utilizing the LLama 3.1 405B model, we generated synthetic data that provided high-quality responses to domain-specific prompts.
3. **Fine-Tuning LLama 3 8B**: We fine-tuned the LLaMA 3 8B model using the synthetic data, adapting it to better handle specific tasks and improving its overall performance.
4. **Model Testing**: We tested the fine-tuned model against a set of evaluation questions, comparing its responses to those of the pre-trained model and assessing the improvements achieved through fine-tuning.

### Results and Insights:
- **Enhanced Performance**: The fine-tuned LLaMA 3 8B model demonstrated significant improvements in generating accurate and contextually relevant responses, showcasing the effectiveness of the fine-tuning process.
- **Cost-Effective Adaptation**: By fine-tuning the smaller 8B model with data generated from the larger 405B model, we achieved high performance without the need for extensive computational resources, highlighting a cost-effective approach to model adaptation.
- **Scalability and Flexibility**: The workflow outlined in this notebook can be scaled and adapted to various domains and tasks, providing a flexible framework for enhancing the capabilities of language models.

### Future Work:
- **Further Fine-Tuning**: Additional fine-tuning with more diverse and extensive datasets can further improve the model's performance and adaptability to different domains.
- **Real-World Applications**: Deploying the fine-tuned model in real-world applications such as customer support, content generation, and domain-specific research can provide valuable insights and practical benefits.
- **Continuous Evaluation**: Ongoing evaluation and monitoring of the model's performance will ensure that it remains effective and relevant as new data and requirements emerge.

In conclusion, this notebook has provided a comprehensive guide to fine-tuning and evaluating the LLama 3 8B model, demonstrating the potential of using advanced language models to address specific domain needs. By the steps outlined, practitioners can enhance their models' performance, achieve cost-effective adaptations, and unlock new possibilities in natural language processing and beyond.

In [139]:
# llama_3_8b_predictor.delete_predictor()

# llama_3_1_405b_predictor.delete_predictor()

# finetuned_predictor.delete_predictor()


[2024-07-19 14:55:59,707] p54 {session.py:4608} INFO - Deleting endpoint configuration with name: meta-textgeneration-llama-3-8b-instruct-2024-07-19-13-38-42-825
[2024-07-19 14:55:59,868] p54 {session.py:4598} INFO - Deleting endpoint with name: meta-textgeneration-llama-3-8b-instruct-2024-07-19-13-38-42-825
[2024-07-19 14:56:00,053] p54 {session.py:4608} INFO - Deleting endpoint configuration with name: meta-textgeneration-llama-3-8b-instruct-2024-07-19-14-34-50-076
[2024-07-19 14:56:01,935] p54 {session.py:4598} INFO - Deleting endpoint with name: meta-textgeneration-llama-3-8b-instruct-2024-07-19-14-34-50-076
